In [7]:
import sys

sys.path.append('../')
import pandas as pd
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
from src.ewenpyomo import ewen_model
from src.neural_network import create_nn, data_scaling
from src.omlt_model import create_model

__OMLT__

In [8]:
# Data loading
df = pd.read_csv('../data/datasetcost.csv')

inputs = list(df.columns)
inputs.remove('total_cost')
outputs = ['total_cost']

print(f'Dataset shape: {df.shape}')

Dataset shape: (10000, 7)


In [9]:
%load_ext autoreload
%autoreload

# Data scaling
x, y, x_offset, x_factor, \
    y_offset, y_factor, scaled_lb, scaled_ub = data_scaling(df, inputs, outputs)
    
# Create and train a ANN
# net = create_nn(x, y)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import tensorflow.keras as keras
from omlt import OffsetScaling, OmltBlock
from omlt.io.keras import load_keras_sequential
from omlt.neuralnet import (
    FullSpaceNNFormulation,
    FullSpaceSmoothNNFormulation,
    NetworkDefinition,
    ReducedSpaceSmoothNNFormulation,
    ReluBigMFormulation,
    ReluComplementarityFormulation,
    ReluPartitionFormulation,
)

omlt_model = pyo.ConcreteModel()
omlt_model.cost = OmltBlock()
nn_cost = keras.models.load_model('cost_nn.keras', compile=False)

scaler = OffsetScaling(
    offset_inputs={i: x_offset[inputs[i]] for i in range(len(inputs))},
    factor_inputs={i: x_factor[inputs[i]] for i in range(len(inputs))},
    offset_outputs={i: y_offset[outputs[i]] for i in range(len(outputs))},
    factor_outputs={i: y_factor[outputs[i]] for i in range(len(outputs))},
)

scaled_input_bounds = {i: (scaled_lb[i], scaled_ub[i]) for i in range(len(inputs))}

net = load_keras_sequential(
    nn_cost, scaling_object=scaler, scaled_input_bounds=scaled_input_bounds
)
omlt_model.cost.build_formulation(FullSpaceSmoothNNFormulation(net))


In [11]:
# Load ewen basic model
ewen_pyomo_model = ewen_model()

# Create merged model
model = pyo.ConcreteModel()
model.model1 = ewen_pyomo_model
model.model2 = omlt_model

In [12]:
cost = outputs.index('total_cost')

# Create objective function
model.obj = pyo.Objective(expr=model.model1.EFTotal + 3 * model.model2.cost.outputs[cost], sense=pyo.minimize)

In [13]:
FiR_1 = inputs.index('Fir1')
FiR_2 = inputs.index('Fir2')
FiR_3 = inputs.index('Fir3')
CiR_1 = inputs.index('Cir1')
CiR_2 = inputs.index('Cir2')
CiR_3 = inputs.index('Cir3')

model.constraint1 = pyo.Constraint(expr=model.model1.FiR[1] == model.model2.cost.inputs[FiR_1])
model.constraint2 = pyo.Constraint(expr=model.model1.FiR[2] == model.model2.cost.inputs[FiR_2])
model.constraint3 = pyo.Constraint(expr=model.model1.FiR[3] == model.model2.cost.inputs[FiR_3])

model.constraint4 = pyo.Constraint(expr=model.model1.CiR[1] == model.model2.cost.inputs[CiR_1])
model.constraint5 = pyo.Constraint(expr=model.model1.CiR[2] == model.model2.cost.inputs[CiR_2])
model.constraint6 = pyo.Constraint(expr=model.model1.CiR[3] == model.model2.cost.inputs[CiR_3])

In [14]:
solver = pyo.SolverFactory('ipopt')
results = solver.solve(model, tee=False)

In [15]:
print(f'EFTotal value: {model.obj()}')
print(f'cost value: {model.model2.cost.outputs[cost].value}')
print(f'ERTotal value: {model.model1.ERTotal()}')
print(f'interc value: {model.model1.interc()}')
print()
print(f'FiR_1 = {model.model1.FiR[1].value}')
print(f'FiR_2 = {model.model1.FiR[2].value}')
print(f'FiR_3 = {model.model1.FiR[3].value}')
print()
print(f'CiR_1 = {model.model1.CiR[1].value}')
print(f'CiR_2 = {model.model1.CiR[2].value}')
print(f'CiR_3 = {model.model1.CiR[3].value}')
print()
print(f'CoR_1 = {model.model1.CoR[1].value}')
print(f'CoR_2 = {model.model1.CoR[2].value}')
print(f'CoR_3 = {model.model1.CoR[3].value}')

EFTotal value: 247.82421611991316
cost value: 40.63065871821597
ERTotal value: 204.9273204905673
interc value: 28.0

FiR_1 = 42.63868150816369
FiR_2 = 26.676358816374773
FiR_3 = 135.6122801660288

CiR_1 = 6395.8022047899185
CiR_2 = 4001.453734710432
CiR_3 = 29999.278766770854

CoR_1 = 852.773630353511
CoR_2 = 266.76358825894545
CoR_3 = 1356.1228017554859


__List model constraints__

In [24]:
# Print all constraints of model.model2
for i in model.model2.cost.component_objects(pyo.Constraint, active=True):
    print(f'Constraint: {i}')
    i.pprint()

Constraint: model2.cost._scale_input_constraint
_scale_input_constraint : Size=6, Index=model2.cost.inputs_set, Active=True
    Key : Lower : Body                                                                                          : Upper : Active
      0 :   0.0 : model2.cost.scaled_inputs[0] - (model2.cost.inputs[0] - 100.84148836101173)/57.75479277168155 :   0.0 :   True
      1 :   0.0 :  model2.cost.scaled_inputs[1] - (model2.cost.inputs[1] - 99.15751764899935)/57.77639965009035 :   0.0 :   True
      2 :   0.0 :  model2.cost.scaled_inputs[2] - (model2.cost.inputs[2] - 100.025267445675)/57.791990694744236 :   0.0 :   True
      3 :   0.0 : model2.cost.scaled_inputs[3] - (model2.cost.inputs[3] - 15102.450670284874)/8658.758243352475 :   0.0 :   True
      4 :   0.0 : model2.cost.scaled_inputs[4] - (model2.cost.inputs[4] - 14977.487027749577)/8632.839765380475 :   0.0 :   True
      5 :   0.0 : model2.cost.scaled_inputs[5] - (model2.cost.inputs[5] - 15061.205277568564)/8690.514